In [10]:
#This pandas library helps to create data frame in the program
import pandas as pd

#This request library helps to fetch the json data from url and process it
import requests as req

#InternalUnwrapper class helps in flattening nested dictionary and list data inside the JSON Data
class InternalUnwrapper:
    
    #This is the constructor of DictUnwrapper Class
    def __init__(self):
        self._data_list = None
    
    #This is the getter method to fetch the data from protected _data_list variable
    def get_data_list(self):
        return self._data_list
    
    #This is the setter method to set the data into protected _data_list variable
    def set_data_list(self, data_list):
        self._data_list = data_list
    
    #This method unwrap_internal_dict_list does the unwrapping for nested dictionary and list inside the JSON data
    def unwrap_internal_dict_list(self, data_series, col):
        
        try:
            
            data_list = []
            for data in data_series:
                data_dict = {}
                if(type(data) == dict):
                    for key, values in data.items():
                        data_dict[str(col)+"."+str(key)] = values
                        if type(values) == dict or type(values) == list:
                            data_dict.update((self.unwrap_internal_dict_list([values], str(col)+"."+str(key)))[0])
                elif(type(data) == list and len(data) >= 1):
                    for i in range(len(data)):
                        data_dict[str(col)+"."+str(i)] = data[i]
                        if type(data[i]) == dict or type(data[i]) == list:
                            data_dict.update((self.unwrap_internal_dict_list([data[i]], str(col)+"."+str(i)))[0])
                
                data_list.append(data_dict)
        
        except Exception as e:
            
            print("Error occured in unwrap_internal_dict_list due to: {} ".format(str(e)))
            
        else:
            
            self.set_data_list(data_list)
            return self.get_data_list()


#JSONUnwrapper class helps in flattening the overall JSON data fed to it and return the final result.
class JSONUnwrapper(InternalUnwrapper):
    
    #This is the constructor of JSONUnwrapper Class
    def __init__(self, url):
        super(JSONUnwrapper, self)
        self._url = url
        self._json_data = None
        self._json_data_frame = None
    
    #This is the getter method to fetch the data from protected _url variable
    def get_url(self):
        return self._url
        
    #This is the setter method to set the data into protected _json_data variable
    def set_json_data(self, json_resp):
        self._json_data = json_resp
    
    #This is the getter method to fetch the data from protected _url variable, creates a json response and returns it.
    def get_json_data(self):
        
        try:
            
            resp = req.get(self.get_url())
            self.set_json_data(resp.json())
            return self._json_data
            
        except Exception as e:
            
            print("Error occured in get_json_data due to: {} ".format(str(e)))

    
    #This is the getter method to fetch the data from protected _json_data_frame variable
    def get_json_data_frame(self):
        return self._json_data_frame
    
    #This is the setter method to set the data into protected _json_data_frame variable
    def set_json_data_frame(self, json_data_frame):
        self._json_data_frame = json_data_frame

    
    #This json_unwrapper is the primary method of this class which unwraps the whole JSON data and converts it into data frame and returns the result.
    def json_unwrapper(self):
        
        try:
            
            json_data_frame = pd.DataFrame(self.get_json_data())
            frame_list = []
            frame_list.append(json_data_frame)
            for col in json_data_frame.keys():
                for data in json_data_frame[col]:
                    if data is not None:
                        if(type(data) == dict or type(data) == list):
                            temp_frame = pd.DataFrame(self.unwrap_internal_dict_list(json_data_frame[col], col))
                            frame_list.append(temp_frame)
                            break
            
            self.set_json_data_frame(pd.concat(frame_list, axis=1)) #pd.concat() is used to merge multiple data frames sequentially.
            return self.get_json_data_frame().fillna("None") #fillna() is used to replace the NaN values in your whole data frame with a value of your choice
            
        except Exception as e:
            
            print("Error occured in json_unwrapper due to: {} ".format(str(e)))


#input
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
result_obj = JSONUnwrapper(url)
result_obj.json_unwrapper()
#result_obj.json_unwrapper().to_csv("Unravelled_DataSet.csv")

,url,repository_url,labels_url,comments_url,events_url,html_url,id,node_id,number,title,...,milestone.closed_issues,milestone.state,milestone.created_at,milestone.updated_at,milestone.due_on,milestone.closed_at,pull_request.url,pull_request.html_url,pull_request.diff_url,pull_request.patch_url
0,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/36283,698880605,MDExOlB1bGxSZXF1ZXN0NDg0NjgxMDQ5,36283,TST: #31922 assert no segmentation fault with ...,...,None,None,None,None,None,None,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/36283,https://github.com/pandas-dev/pandas/pull/3628...,https://github.com/pandas-dev/pandas/pull/3628...
1,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/36282,698753726,MDExOlB1bGxSZXF1ZXN0NDg0NTY3Nzc0,36282,REF: de-duplicate _wrap_joined_index,...,None,None,None,None,None,None,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/36282,https://github.com/pandas-dev/pandas/pull/3628...,https://github.com/pandas-dev/pandas/pull/3628...
2,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/issues/36281,698751134,MDU6SXNzdWU2OTg3NTExMzQ=,36281,QST: Appending to pandas DataFrames,...,None,None,None,None,None,None,None,None,None,None
3,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/36280,698713545,MDExOlB1bGxSZXF1ZXN0NDg0NTMxMzQ1,36280,PERF: CategoricalDtype.__eq__,...,None,None,None,None,None,None,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/36280,https://github.com/pandas-dev/pandas/pull/3628...,https://github.com/pandas-dev/pandas/pull/3628...
4,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/issues/36279,698560986,MDU6SXNzdWU2OTg1NjA5ODY=,36279,QST: Python 3.9 testing in CI,...,None,None,None,None,None,None,None,None,None,None
5,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/36278,698528739,MDExOlB1bGxSZXF1ZXN0NDg0MzYyMjYz,36278,[TST]: Add test for 24768,...,None,None,None,None,None,None,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/36278,https://github.com/pandas-dev/pandas/pull/3627...,https://github.com/pandas-dev/pandas/pull/3627...
6,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/36277,698509860,MDExOlB1bGxSZXF1ZXN0NDg0MzQ1MTY4,36277,ENH: add peek function,...,None,None,None,None,Non